In [1]:
import numpy as np
import pickle
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.models import load_model
from keras import regularizers
from keras.layers.core import Dropout
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix as cm
from keras.metrics import Precision
from keras.metrics import Recall
import cv2 as cv
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [ ]:
import pickle
def DataExpansion(a):
    with open(a, 'rb') as f:
      l = pickle.load(f, encoding='bytes')
      f.close()
      
    
    training_data = np.array(l[0][0])
    training_data_out = np.array(l[0][1])
    test_data = np.array(l[1][0])
    test_data_out = np.array(l[1][1])
    
    training_data = training_data.reshape(training_data.shape[0],224,224,1)
    test_data = test_data.reshape(test_data.shape[0],224,224,1)
#    training_data = training_data.astype('float32')
#    test_data = test_data.astype('float32')
    number_of_classes = 2
    from keras.utils import np_utils
    training_data_out = np_utils.to_categorical(training_data_out, number_of_classes)
    test_data_out = np_utils.to_categorical(test_data_out, number_of_classes)
    training_data_out = training_data_out[:,0]
    test_data_out = test_data_out[:,0]
    
    return training_data,training_data_out,test_data,test_data_out
          
def data():
  X_train = pd.read_csv('/content/drive/My Drive/X_train1.csv')
  y_train = pd.read_csv('/content/drive/My Drive/Y_train1.csv')
  X_test = pd.read_csv('/content/drive/My Drive/X_test_all.csv')
  y_test = pd.read_csv('/content/drive/My Drive/Y_test_all.csv')
  X_train = X_train.iloc[:,1:].values
  X_test = X_test.iloc[:,1:].values
  y_train = y_train.iloc[:,1:].values
  y_test = y_test.iloc[:,1:].values
  X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.2, random_state=12345)
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')
  X_val = X_val.astype('float32')
  nb_classes = 12
  y_train = np_utils.to_categorical(y_train, nb_classes)
  y_val = np_utils.to_categorical(y_val, nb_classes)
  y_test= np_utils.to_categorical(y_test, nb_classes)
  y_train = y_train[:,0:11]
  y_test = y_test[:,0:11]
  y_val = y_val[:,0:11]
  X_train = X_train.reshape(X_train.shape[0],224,224,1)
  X_test = X_test.reshape(X_test.shape[0],224,224,1)
  X_val = X_val.reshape(X_val.shape[0],224,224,1)
  return X_train, y_train, X_val, y_val

def model():
    classifier = Sequential()
    #convolution layer with rectifier function
    # input_shape = (64,64,1) for 'tensorflow backend' and (1,64,64) for 'theano backend'
    classifier.add(Conv2D(32,(3,3),input_shape = (224,224,1),activation = 'relu'))
    #classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size = (2, 2)))
    classifier.add(Conv2D(64,(3,3),activation = 'relu'))
    #classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size = (2, 2)))
    classifier.add(Conv2D(128,(3,3),activation = 'relu'))
    #classifier.add(BatchNormalization())
    #classifier.add(MaxPooling2D(pool_size = (2, 2)))
    classifier.add(Conv2D(128,(3,3),activation = 'relu'))
    #classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size = (2, 2)))
    classifier.add(Conv2D(256,(3,3),activation = 'relu'))
    #classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size = (2, 2)))
    classifier.add(Flatten())
    classifier.add(Dense(units =512,activation = 'relu')
    classifier.add(Dense(units = 1,activation = 'sigmoid'))
    classifier.summary()
    classifier.compile(optimizer = 'adam',loss = 'binary_crossentropy', metrics = ['accuracy','Precision','Recall'])  
    
    return classifier

def model_2():
  classifier = Sequential()
  classifier.add(Conv2D(32,3,3,input_shape = (224,224,1),activation = 'relu'))
  #classifier.add(BatchNormalization())
  classifier.add(MaxPooling2D(pool_size = (2, 2)))

  classifier.add(Conv2D(64,3,3,activation = 'relu'))
  #classifier.add(BatchNormalization())
  classifier.add(MaxPooling2D(pool_size = (2, 2)))

  classifier.add(Conv2D(128,3,3,activation = 'relu'))
  #classifier.add(BatchNormalization())
  classifier.add(MaxPooling2D(pool_size = (2, 2)))
  classifier.add(Conv2D(128,3,3,activation = 'relu'))
  #classifier.add(BatchNormalization())
  classifier.add(MaxPooling2D(pool_size = (2, 2)))

  classifier.add(Conv2D(256,3,3,activation = 'relu'))
  # classifier.add(BatchNormalization())
  #classifier.add(Conv2D(32,3,3,activation = 'relu'))
  # classifier.add(BatchNormalization())
  classifier.add(MaxPooling2D(pool_size = (2, 2)))

  classifier.add(Flatten())
  classifier.add(Dense(units =512,activation = 'relu'))
  classifier.add(Dense(units = 1,activation = 'sigmoid'))
  classifier.summary()
  classifier.compile(optimizer = 'adam',loss = 'categorical_crossentropy', metrics = ['categorical_accuracy','Precision','Recall'])

  return classifier

   
def run_model(model,trd,trdo,tsd,tsdo,n_ep):
    train_datagen = ImageDataGenerator(rescale=1./255,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True)
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    training_set = train_datagen.flow(trd,trdo,batch_size=32)
                                                     
    test_set = test_datagen.flow(tsd,tsdo,batch_size=32)
                                                
    model.fit_generator(training_set,
                        steps_per_epoch=trdo.shape[0],
                        epochs=n_ep,
                        validation_data=test_set,
                        validation_steps=tsdo.shape[0],
                        callbacks = [EarlyStopping(monitor='val_acc', patience=3)])
    return model
    



In [ ]:
a,b,c,d = DataExpansion('/content/drive/My Drive/MODS_all_data_bw_224_224_0.pkl')

In [ ]:
for i in range(1,5):
    if i == 1:
        run_model(model_2(),a,b,c,d,15).save('binary_class_'+str(i)+'.h5')
    else:
        mod = load_model('/content/binary_class_'+str(i-1)+'.h5')
        run_model(mod,a,b,c,d,15).save('/content/binary_class_'+str(i)+'.h5')

In [ ]:
a,b,c,d = data()

In [ ]:
for i in range(1,5):
    if i == 1:
        run_model(model_2(),a,b,c,d,15).save('multi_class_'+str(i)+'.h5')
    else:
        mod = load_model('/content/multi_class_'+str(i-1)+'.h5')
        run_model(mod,a,b,c,d,15).save('/content/multi_class_'+str(i)+'.h5')